In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple
from itertools import combinations, permutations
import random



class SingleMachineScheduling:
    def __init__(self, n_tasks: int):
        self.n_tasks = n_tasks
        self.tasks = self.generate_tasks()

    def greedy(self, tasks: pd.DataFrame, criterion: str) -> pd.DataFrame:
        if criterion == "SPT":  # shortest processing time; optimal solution to minimize elapsed waiting time 
            sort_prio = ["processing_time", "due_date"]
        elif criterion == "EDD":  # earliest due date; provides optimal solution to minimize maximum task tardiness
            sort_prio = ["due_date", "processing_time"]
        else:
            raise NotImplementedError(f"{criterion} is not a valid greedy priorization criterion")
        current_time = 0
        for i in range(1, len(tasks) + 1):
            tasks = tasks.sort_values(by=sort_prio)
            next_task_id = tasks[~tasks["done"]].index[0]
            tasks, current_time = self._execute_task(tasks, next_task_id, current_time, i)
        return tasks.sort_values(by="execution_order")

    def generate_tasks(self) -> pd.DataFrame:
        due_date = np.random.randint(low=5, high=30, size=self.n_tasks)
        processing_time = np.maximum(1, due_date - np.random.randint(low=0, high=5, size=self.n_tasks))
        importance = np.random.randint(low=1, high=6, size=self.n_tasks)
        return pd.DataFrame({
            "id": range(self.n_tasks),
            "due_date": due_date,
            "processing_time": processing_time,
            "importance": importance,
            "done": False,
            "tardiness": 0,
            "execution_order": 0,
            "completion_time": 0})
    
    def execute_tasks(self, tasks: pd.DataFrame, execution_order: List[int]):
        current_time = 0
        for i, task_id in enumerate(execution_order):
            tasks, current_time = self._execute_task(tasks=tasks, task_id=task_id, current_time=current_time, order=i)
        return tasks

    def _execute_task(self, tasks: pd.DataFrame, task_id: int, current_time: int, order: int) -> tuple[pd.DataFrame, int]:
        task = tasks.loc[task_id]
        start_time = current_time
        completion_time = start_time + task["processing_time"]
        tardiness = max(0, completion_time - task["due_date"])

        tasks.loc[task_id, "done"] = True
        tasks.loc[task_id, "tardiness"] = tardiness
        tasks.loc[task_id, "execution_order"] = order
        tasks.loc[task_id, "completion_time"] = completion_time
        return tasks, completion_time
    
    def _total_tardiness(self, execution_oder: List[int]) -> int:
        return int(self.execute_tasks(tasks=self.tasks, execution_order=execution_oder)["tardiness"].sum())





class BranchAndBound(SingleMachineScheduling):
    def __init__(self, n_tasks: int):
        super().__init__(n_tasks=n_tasks)

    def run(self, verbose: bool = False):
        """
        first greedily explores one path in depth, then tries all other paths while pruning those 
        with a total tardiness lower bound bigger than the greedy total tardiness
        """
        candidates = list(permutations(self.tasks["id"].to_list()))
        best_known_attempt = self.greedy(tasks=self.tasks, criterion="SPT")["id"].to_list()
        best_known_total_tardiness = self._total_tardiness(execution_oder=best_known_attempt)
        if verbose: print(f"greedy solution: {best_known_attempt} ->  total tardiness {best_known_total_tardiness}")
        for stage in range(1, len(self.tasks)):  # all but very last stage
            candidates_stage = list(set([c[:stage] for c in candidates]))
            if verbose: print(f"stage {stage}: {candidates_stage}")
            for candidate_stage in candidates_stage:
                candidate_stage = [i for i in candidate_stage]
                candidate_lower_bound = self.lower_bound(fixed_execution_order=candidate_stage)
                printstr = f"{candidate_stage} -> lower bound: {candidate_lower_bound}"
                if candidate_lower_bound >= best_known_total_tardiness:
                    printstr += f" -> prune subtree beginning at {candidate_stage}"
                    candidates = [c for c in candidates if c[:stage] != tuple(candidate_stage)]
                if verbose: print(printstr)
        if len(candidates) > 0:
            if verbose: print(f"last stage: {candidates}")
            for candidate_last_stage in candidates:
                candidate_total_tardiness = self._total_tardiness(execution_oder=list(candidate_last_stage))
                if verbose: print(f"{candidate_last_stage} -> total tardiness: {candidate_total_tardiness}")
                if candidate_total_tardiness < best_known_total_tardiness:
                    best_known_attempt = candidate_last_stage
                    best_known_total_tardiness = candidate_total_tardiness
        if verbose: print(f"optimal solution: {best_known_attempt}")
        return best_known_total_tardiness

    def lower_bound(self, fixed_execution_order: List[int]) -> int:
        """
        fixed path: total tardiness of fixed path 
        child subtree: current total tardiness of open tasks after absolving fixed path (always <= full path total tardiness)  
        """
        fixed_tasks = self.tasks[self.tasks["id"].isin(fixed_execution_order)]
        open_tasks = self.tasks[~self.tasks["id"].isin(fixed_execution_order)]
        executed_fixed_tasks = self.execute_tasks(tasks=fixed_tasks, execution_order=fixed_execution_order)
        open_tasks = open_tasks.copy()
        open_tasks["processing_time"] += fixed_tasks["processing_time"].sum()
        open_tasks["min_tardiness"] = open_tasks.apply(lambda row: max(0, row["processing_time"] - row["due_date"]), axis=1)
        return open_tasks["min_tardiness"].sum() + executed_fixed_tasks["tardiness"].sum()




class TabuSearch(SingleMachineScheduling):
    def __init__(self, n_iters: int, n_tasks: int, max_len_swap_tabu_list: int, max_len_candidate_tabu_list: int, neighborhood_sample_size: int = 5, greedy_criterion: str = "due_date"):       
        super().__init__(n_tasks=n_tasks) 
        self.max_len_swap_tabu_list = max_len_swap_tabu_list
        self.max_len_candidate_tabu_list = max_len_candidate_tabu_list
        self.n_iters = n_iters
        self.n_tasks = n_tasks
        self.neighborhood_sample_size = neighborhood_sample_size

        self.tasks = self.generate_tasks()
        result_greedy =  self.greedy(tasks=self.tasks, criterion=greedy_criterion)  # search trajectory starting point
        self.current_best_solution = result_greedy["id"].to_list()
        self.swap_tabu_list = []
        if max_len_candidate_tabu_list > 0:
            self.candidate_tabu_list = [self.current_best_solution]
        else:
            self.candidate_tabu_list = []

    def run(self):
        greedy_cost = self._total_tardiness(execution_oder=self.current_best_solution)
        current_best_cost = greedy_cost
        for _ in range(self.n_iters):
            neighbors = self.neighborhood()
            # tabu here
            for neighbor in neighbors:
                cost_neighbor = self._total_tardiness(execution_oder=neighbor) 
                if cost_neighbor < current_best_cost:
                    current_best_cost = cost_neighbor
                    self.current_best_solution = neighbor
        return greedy_cost, current_best_cost

    def neighborhood(self):
        """
        creates neighborhood with swaps and neighbors that don't violate tabu lists
        """
        possible_swaps = list(combinations(range(len(self.current_best_solution)), 2))  # contains (0, 1) but not (1, 0) etc. doesn't contain (0, 0)
        possible_swaps = [swap for swap in possible_swaps if swap not in self.swap_tabu_list]
        neighborhood = []
        swaps = []
        for _ in range(self.neighborhood_sample_size * 5):  # max iters
            swap = possible_swaps.pop()
            swap0_val = self.current_best_solution[swap[0]]
            neighbor = self.current_best_solution.copy()
            neighbor[swap[0]] = neighbor[swap[1]]
            neighbor[swap[1]] = swap0_val
            if neighbor not in self.candidate_tabu_list: 
                neighborhood.append(neighbor)
                swaps.append(swap)
                if len(neighborhood) == self.neighborhood_sample_size:
                    break
        self._update_swap_tabu(new_swaps=swaps)
        self._update_candidate_tabu_list(new_candidates=neighborhood)
        return neighborhood
    
    def _update_candidate_tabu_list(self, new_candidates: List[List[int]]):
        # random.sample: without replacement
        to_free = max(0, (len(self.swap_tabu_list) + len(new_candidates)) - self.max_len_candidate_tabu_list)
        self.candidate_tabu_list = self.candidate_tabu_list[to_free:] + random.sample(population=new_candidates, k=min(len(new_candidates), self.max_len_candidate_tabu_list))  # fifo        

    def _update_swap_tabu(self, new_swaps: List[Tuple[int, int]]):
        # random.sample: without replacement
        to_free = max(0, (len(self.swap_tabu_list) + len(new_swaps)) - self.max_len_swap_tabu_list)
        self.swap_tabu_list = self.swap_tabu_list[to_free:] + random.sample(population=new_swaps, k=min(len(new_swaps), self.max_len_candidate_tabu_list))  # fifo


greedy_costs = []
tabu_best_costs = []
bb_costs = []
for i in range(100):
    tabu_search = TabuSearch(n_iters=5, n_tasks=5, max_len_candidate_tabu_list=5, max_len_swap_tabu_list=5, neighborhood_sample_size=5, greedy_criterion="SPT")
    bb = BranchAndBound(n_tasks=5)
    bb.tasks = tabu_search.tasks.copy()
    greedy_cost, tabu_best_cost = tabu_search.run()
    bb_costs.append(bb.run())
    greedy_costs.append(greedy_cost)
    tabu_best_costs.append(tabu_best_cost)
df_result = pd.DataFrame({"total tardiness greedy (SPT)": greedy_costs, "total tardiness tabu": tabu_best_costs, "total tardiness branch & bound": bb_costs})
df_result[df_result["total tardiness greedy (SPT)"] > df_result["total tardiness branch & bound"]]

,total tardiness greedy (SPT),total tardiness tabu,total tardiness branch & bound
9,136,133,133
17,83,82,82
21,68,67,67
37,53,51,51
46,93,90,90
65,73,72,72
78,156,155,155
97,65,64,64


In [2]:
bb = BranchAndBound(n_tasks=3)
_ = bb.run(verbose=True)

initial total tardiness 16
stage 1: [(0,), (1,), (2,)]
[0] -> lower bound: 8
[1] -> lower bound: 27 -> prune subtree beginning at [1]
[2] -> lower bound: 9
stage 2: [(0, 1), (0, 2), (2, 0), (2, 1)]
[0, 1] -> lower bound: 23 -> prune subtree beginning at [0, 1]
[0, 2] -> lower bound: 14
[2, 0] -> lower bound: 16 -> prune subtree beginning at [2, 0]
[2, 1] -> lower bound: 24 -> prune subtree beginning at [2, 1]
last stage: [(0, 2, 1)]
(0, 2, 1) -> total tardiness: 14
optimal solution: (0, 2, 1)
